# CAT12 Parcellation Example

This notebook demonstrates how to use the CAT12 interface to parcellate gray matter (GM), white matter (WM), and cortical thickness (CT) maps.

## Expected CAT12 Directory Structure

The interface expects CAT12 outputs organized in a BIDS-like structure:

```
cat12_derivatives/
  sub-01/
    ses-01/  (optional)
      anat/
        mwp1sub-01_ses-01_T1w.nii.gz  # GM probability map
        mwp2sub-01_ses-01_T1w.nii.gz  # WM probability map
        wctT1w.nii.gz                  # Cortical thickness
```

## File Naming Conventions

- `mwp1*` - Modulated warped gray matter probability maps
- `mwp2*` - Modulated warped white matter probability maps  
- `wct*` - Warped cortical thickness maps

## 1. Setup and Configuration

In [ ]:
from pathlib import Path

from parcellate.interfaces.cat12 import (
    AtlasDefinition,
    Cat12Config,
    TissueType,
    run_parcellations,
)
from parcellate.interfaces.cat12.loader import TISSUE_PATTERNS

In [1]:
import pandas as pd

df = pd.read_csv(
    "/media/storage/yalab-dev/BIDS/derivatives/cat12_parcellated/cat12/sub-CLMC10/ses-202407110849/anat/atlas-Schaefer2018N400n7Tian2020S3/sub-CLMC10_ses-202407110849_atlas-Schaefer2018N400n7Tian2020S3_space-MNI152NLin2009cAsym_res-01_tissue-GM_mask-gm_maskthr-50_parc.tsv",
    sep="\t",
)
df

,index,label,network_label,label_7network,index_17network,label_17network,network_label_17network,atlas_name,network_id,volume_mm3,...,percentile_50,percentile_75,percentile_90,percentile_95,is_strongly_skewed,is_heavy_tailed,is_bimodal,has_outliers,fails_normality,vol_TIV
0,1,LH_Vis_1,Vis,7Networks_LH_Vis_1,61.0,17Networks_LH_DorsAttnA_TempOcc_2,DorsAttnA,Schaefer2018N400n7,NaN,1851.773030,...,0.771793,0.865347,0.937192,0.960640,True,False,False,False,True,1522.940773
1,2,LH_Vis_2,Vis,7Networks_LH_Vis_2,193.0,17Networks_LH_DefaultC_PHC_2,DefaultC,Schaefer2018N400n7,NaN,2045.228622,...,0.730135,0.854066,0.896783,0.916487,True,False,False,False,True,1522.940773
2,3,LH_Vis_3,Vis,7Networks_LH_Vis_3,1.0,17Networks_LH_VisCent_ExStr_1,VisCent,Schaefer2018N400n7,NaN,1626.672533,...,0.771299,0.862507,0.933541,0.964859,True,False,False,False,True,1522.940773
3,4,LH_Vis_4,Vis,7Networks_LH_Vis_4,13.0,17Networks_LH_VisPeri_ExStrInf_1,VisPeri,Schaefer2018N400n7,NaN,1953.171505,...,0.765492,0.852241,0.909348,0.939979,True,False,False,False,True,1522.940773
4,5,LH_Vis_5,Vis,7Networks_LH_Vis_5,2.0,17Networks_LH_VisCent_ExStr_2,VisCent,Schaefer2018N400n7,NaN,1687.379247,...,0.704999,0.850362,0.920313,0.949334,True,False,False,False,True,1522.940773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,446,THA-DP-lh,Subcortex,NaN,NaN,NaN,NaN,Tian2020S3,NaN,906.132148,...,0.712676,0.811634,0.865560,0.883970,True,False,False,False,True,1522.940773
446,447,NAc-shell-lh,Subcortex,NaN,NaN,NaN,NaN,Tian2020S3,NaN,917.940408,...,0.787919,0.834184,0.859190,0.867213,True,True,False,True,True,1522.940773
447,448,NAc-core-lh,Subcortex,NaN,NaN,NaN,NaN,Tian2020S3,NaN,568.790274,...,0.799949,0.860654,0.887692,0.900222,True,True,False,True,True,1522.940773
448,449,pGP-lh,Subcortex,NaN,NaN,NaN,NaN,Tian2020S3,NaN,4.747739,...,0.443578,0.513327,0.555176,0.569126,False,True,False,False,False,1522.940773


In [21]:
# Set your paths here
DATA_ROOT = Path("/media/storage/yalab-dev/BIDS/derivatives/CAT12.9_2577.new")  # CAT12 output directory
ATLAS_PATH = Path(
    "/mnt/62/Processed_Data/derivatives/qsirecon/atlases/atlas-4S456Parcels/atlas-4S456Parcels_space-MNI152NLin2009cAsym_res-01_dseg.nii.gz"
)  # Your atlas file
ATLAS_LUT = Path(
    "/mnt/62/Processed_Data/derivatives/qsirecon/atlases/atlas-4S456Parcels/atlas-4S456Parcels_dseg.tsv"
)  # Atlas lookup table (optional)
OUTPUT_DIR = Path("/media/storage/yalab-dev/BIDS/derivatives/cat12_parcellated")  # Where to save parcellation results

In [24]:
import pandas as pd

df = pd.read_csv("~/Downloads/linked_sessions.csv")
df_test = df[df["subject_code"].astype(str).str.zfill(4).isin(["0355", "0350", "0348"])]
df_test.to_csv("~/Downloads/linked_sessions_test.csv", index=False)

## 2. Method 1: Using a TOML Configuration File

The simplest way to run CAT12 parcellation is via a TOML configuration file.

### Example TOML Configuration

Create a file named `cat12_config.toml`:

```toml
input_root = "/path/to/cat12/derivatives"
output_dir = "/path/to/output"
subjects = ["01", "02"]  # Optional: process specific subjects
sessions = ["01"]         # Optional: process specific sessions
force = false             # Set to true to overwrite existing outputs
log_level = "INFO"

[[atlases]]
name = "Schaefer400"
path = "/path/to/atlas/schaefer400.nii.gz"
lut = "/path/to/atlas/schaefer400.tsv"
space = "MNI152NLin2009cAsym"

[[atlases]]
name = "AAL"
path = "/path/to/atlas/aal.nii.gz"
space = "MNI152NLin2009cAsym"
```

In [16]:
# Load config from TOML and run parcellations
# config = load_config(Path("/home/galkepler/Projects/parcellate/cat12_config.toml"))
# output_files = run_parcellations(config)
# print(f"Generated {len(output_files)} parcellation files")

## 3. Method 2: Programmatic Configuration

You can also configure the parcellation workflow programmatically.

In [17]:
# Define atlases programmatically
atlases = [
    AtlasDefinition(
        name="4S456Parcels",
        nifti_path=ATLAS_PATH,
        lut=ATLAS_LUT,
        space="MNI152NLin2009cAsym",
    ),
    # Add more atlases as needed
]

In [18]:
# Create configuration
config = Cat12Config(
    input_root=DATA_ROOT,
    output_dir=OUTPUT_DIR,
    atlases=atlases,
    subjects=["0350"],  # None = process all subjects
    sessions=["202012151539"],  # None = process all sessions
    force=False,  # Don't overwrite existing outputs
)

print(f"Input root: {config.input_root}")
print(f"Output dir: {config.output_dir}")
print(f"Atlases: {[a.name for a in config.atlases]}")

Input root: /media/storage/yalab-dev/BIDS/derivatives/CAT12.9_2577.new
Output dir: /media/storage/yalab-dev/BIDS/derivatives/cat12_parcellated
Atlases: ['4S456Parcels']


In [19]:
# Run the full parcellation workflow
output_files = run_parcellations(config)
# print(f"Generated {len(output_files)} parcellation files")

2026-01-22 16:18:51,882 [INFO] Loading CAT12 inputs from /media/storage/yalab-dev/BIDS/derivatives/CAT12.9_2577.new
2026-01-22 16:19:12,437 [INFO] Finished writing 3 parcellation files


## 4. Method 3: Step-by-Step Workflow

For more control, you can run each step of the workflow manually.

In [7]:
# Step 1: Discover CAT12 inputs
# recon_inputs = load_cat12_inputs(
#     root=DATA_ROOT,
#     atlases=atlases,
#     subjects=["01"],  # Optional: specific subjects
# )
#
# print(f"Found {len(recon_inputs)} subject/session combinations")
# for ri in recon_inputs:
#     print(f"  - {ri.context.label}: {len(ri.scalar_maps)} scalar maps")

In [8]:
# Inspect discovered scalar maps
# recon = recon_inputs[0]
# for sm in recon.scalar_maps:
#     print(f"  {sm.tissue_type.value}: {sm.nifti_path.name}")

In [9]:
# Step 2: Plan the parcellation workflow
# plan = plan_cat12_parcellation_workflow(recon)
#
# for atlas, scalar_maps in plan.items():
#     print(f"Atlas: {atlas.name}")
#     for sm in scalar_maps:
#         print(f"  - {sm.tissue_type.value}: {sm.name}")

In [10]:
# Step 3: Run the parcellation workflow
# results = run_cat12_parcellation_workflow(
#     recon=recon,
#     plan=plan,
#     config=config,
# )
#
# print(f"Generated {len(results)} parcellation outputs")

In [11]:
# Inspect results
# result = results[0]
# print(f"Subject: {result.context.label}")
# print(f"Atlas: {result.atlas.name}")
# print(f"Tissue type: {result.scalar_map.tissue_type.value}")
# print(f"\nStats table shape: {result.stats_table.shape}")
# result.stats_table.head()

## 5. Output Structure

The CAT12 interface outputs parcellation results in the following structure:

```
output_dir/
  cat12/
    sub-01/
      ses-01/
        anat/
          atlas-Schaefer400/
            sub-01_ses-01_atlas-Schaefer400_space-MNI152NLin2009cAsym_tissue-GM_parc.tsv
            sub-01_ses-01_atlas-Schaefer400_space-MNI152NLin2009cAsym_tissue-WM_parc.tsv
            sub-01_ses-01_atlas-Schaefer400_space-MNI152NLin2009cAsym_tissue-CT_parc.tsv
```

Each `.tsv` file contains regional statistics (mean, std, volume, etc.) for each atlas region.

## 6. Working with Tissue Types

The CAT12 interface classifies scalar maps by tissue type:

In [ ]:
# Available tissue types
print("Available tissue types:")
for tt in TissueType:
    print(f"  - {tt.value}: {tt.name}")

# File patterns for each tissue type

print("\nFile patterns:")
for tissue_type, pattern in TISSUE_PATTERNS.items():
    print(f"  - {tissue_type.value}: {pattern}")

Available tissue types:
  - GM: GM
  - WM: WM
  - CT: CT

File patterns:
  - GM: mwp1*
  - WM: mwp2*
  - CT: wct*


In [13]:
# Filter results by tissue type
# gm_results = [r for r in results if r.scalar_map.tissue_type == TissueType.GM]
# wm_results = [r for r in results if r.scalar_map.tissue_type == TissueType.WM]
# ct_results = [r for r in results if r.scalar_map.tissue_type == TissueType.CT]
#
# print(f"GM parcellations: {len(gm_results)}")
# print(f"WM parcellations: {len(wm_results)}")
# print(f"CT parcellations: {len(ct_results)}")

## 7. Command-Line Interface

You can also run CAT12 parcellation from the command line:

```bash
python -m parcellate.interfaces.cat12.cat12 cat12_config.toml
```